Test model llama3

In [1]:
%pip install -q python-dotenv~=1.0.1 lamini==3.0.5

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [86]:
import logging
import os
from datetime import datetime
from pprint import pprint
import sqlite3
import pandas as pd
from utils_llm import setup_logging, get_schema, make_llama_3_prompt
import random
import json
import jsonlines
from tqdm import tqdm
from typing import AsyncIterator, Iterator, Union
from lamini.generation.base_prompt_object import PromptObject
from lamini.generation.generation_node import GenerationNode
from lamini.generation.generation_pipeline import GenerationPipeline

In [1]:
from dotenv import load_dotenv
_ = load_dotenv() #load environmental variable LAMINI_API_KEY with key from .env file

import lamini
# Token access to https://app.lamini.ai/tune/
lamini.api_key = "ad3d62932bc25b48963dd3413dd94cb3414dd650fc47f735704badeb56cdf9ca" # Active API Tokens

In [2]:
# Load the llm3 model
llm = lamini.Lamini(model_name="meta-llama/Meta-Llama-3-8B-Instruct")

In [4]:
# Create prompt with template of llm3 model
prompt = """\
<|begin_of_text|><|start_header_id|>system<|end_header_id|>

You are a helpful assistant.<|eot_id|><|start_header_id|>user<|end_header_id|>

Please write a graduate card for my good friend Mạnh Tiền\
<|eot_id|><|start_header_id|>assistant<|end_header_id|>

"""

# Return result of question by llm3 model
result = llm.generate(prompt, max_new_tokens=200)
print(result)

Here's a possible graduate card for your friend Mạnh Tiền:

"Dear Mạnh Tiền,

Huge congratulations on your graduation!

You've worked hard to achieve this milestone, and it's an incredible accomplishment. I'm so proud of you for pushing through challenges and staying committed to your goals.

As you move forward, remember that this is just the beginning of an exciting new chapter in your life. You have so much to look forward to, and I have no doubt that you'll make the most of every opportunity that comes your way.

Here's to many more adventures, successes, and memories to come! Keep shining your light and making a difference in the world.

Best wishes,
[Your Name]"

Feel free to modify it as you see fit to make it more personal and special for your friend Mạnh Tiền!


In [9]:
prompt2 = (
  "<|begin_of_text|>"  # Start of prompt
  "<|start_header_id|>system<|end_header_id|>\n\n"  #  header - system
  "You are a helpful assistant."  # system prompt
  "<|eot_id|>" # end of turn
  "<|start_header_id|>user<|end_header_id|>\n\n" # header - user
  "Please write a graduate card for my good friend Mạnh Tiền"
  "<|eot_id|>" # end of turn
  "<|start_header_id|>assistant<|end_header_id|>\n\n" # header - assistant
)

prompt = prompt2

# Return result of question by llm3 model
result = llm.generate(prompt, max_new_tokens=200)
print(result)

Here's a birthday card message for your friend Mạnh Tiền:

"Dear Mạnh Tiền,

Happy birthday to an amazing friend like you! On your special day, I want to take a moment to express my gratitude for the incredible memories we've shared and the laughter we've enjoyed together. You bring so much joy and positivity to those around you, and I feel lucky to have you in my life.

Here's to another year of adventure, good health, and making more unforgettable memories together! May your birthday be as bright and wonderful as you are.

Wishing you a day filled with love, laughter, and all your favorite things.

Best regards, [Your Name]"

Feel free to modify it as you see fit!


In [11]:
system_prompt = "You are a helpful assistant." # system prompt
user_prompt = "Please write a graduate card for my good friend Mạnh Tiền"

prompt3 = make_llama_3_prompt(user_prompt, system_prompt)
prompt = prompt3

# Return result of question by llm3 model
result = llm.generate(prompt, max_new_tokens=200)
print(result)

Here's a possible graduate card for your friend Mạnh Tiền:

"Dear Mạnh Tiền,

Huge congratulations on your graduation!

You've worked hard to achieve this milestone, and it's an incredible accomplishment. I'm so proud of you for pushing through challenges and staying committed to your goals.

As you move forward, remember that this is just the beginning of an exciting new chapter in your life. You have so much to look forward to, and I have no doubt that you'll make the most of every opportunity that comes your way.

Here's to many more adventures, successes, and memories to come! Keep shining your light and making a difference in the world.

Best wishes,
[Your Name]"

Feel free to modify it as you see fit to make it more personal and special for your friend Mạnh Tiền!


In [13]:
user_prompt = "Tell me a joke about birthday cake"
prompt = make_llama_3_prompt(user_prompt)

result = llm.generate(prompt, max_new_tokens=200)
print(result)

Why was the birthday cake in a bad mood?

Because it was feeling crumby!


Create a SQL Agent

In [10]:
logger = logging.getLogger(__name__)
print(logger)

engine = sqlite3.connect("./nba_roster.db")
print(engine)
# # Create a cursor object using the connection
# cursor = engine.cursor()

# # Sample data template
# teams = ['Los Angeles Lakers', 'Golden State Warriors', 'Brooklyn Nets', 'Milwaukee Bucks', 'Dallas Mavericks']
# names = ['Player ' + str(i) for i in range(1, 101)]
# positions = ['PG', 'SG', 'SF', 'PF', 'C']
# colleges = ['College A', 'College B', 'College C', 'College D', 'College E']

# # Function to generate random salary
# def generate_salary():
#     return f"${random.randint(1000000, 50000000):,}"

# # Generating 100 rows of data
# nba_data = [
#     (
#         random.choice(teams),
#         name,
#         str(random.randint(0, 99)),  # Jersey number
#         random.choice(positions),
#         random.randint(19, 40),  # Age
#         f'{random.randint(6, 7)}-{random.randint(0, 11)}',  # Height (e.g., 6-9)
#         str(random.randint(190, 260)),  # Weight
#         random.choice(colleges),
#         generate_salary()
#     )
#     for name in names
# ]

# # Insert the generated data into the nba_roster table
# cursor.executemany('''
# INSERT INTO nba_roster (team, name, jersey, pos, age, ht, wt, college, salary)
# VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)
# ''', nba_data)

# # Commit the changes
# engine.commit()
# logger.info(f"Connected to the database at './film_database.db': {engine}")

setup_logging()

<Logger __main__ (WARNING)>


In [19]:
user = """Who is the highest paid NBA player?"""

system = f"""You are an NBA analyst with 15 years of experience writing complex SQL queries. Consider the nba_roster table with the following schema:
{get_schema()}

Write a sqlite query to answer the following question. Follow instructions exactly"""

prompt = make_llama_3_prompt(user, system)

print(llm.generate(prompt, max_new_tokens=200))

To answer this question, we can use the following SQLite query:

```sql
SELECT NAME, SALARY
FROM nba_roster
WHERE SALARY!= '--'
ORDER BY CAST(SALARY AS REAL) DESC
LIMIT 1;
```

This query first filters out the rows where the salary is '--' (i.e., the players who don't have a salary listed). Then, it orders the remaining rows by the salary in descending order (highest to lowest). Finally, it limits the result to the top 1 row, which corresponds to the highest paid NBA player.


In [22]:
# We'd like to be able to get just SQL output so we don't have to parse the query from the model response. For this we can use structured output.
result = llm.generate(prompt, output_type={"sqlite_query": "str"}, max_new_tokens=200)
print(result)

# Query data from model results
df = pd.read_sql(result['sqlite_query'], con=engine)
df

{'sqlite_query': "SELECT * FROM nba_roster WHERE SALARY!= '--' ORDER BY CAST(SALARY AS REAL) DESC LIMIT 1"}


,id,team,name,jersey,pos,age,ht,wt,college,salary
0,1,Brooklyn Nets,Player 1,18,SF,31,7-2,211,College D,"$17,565,125"


In [13]:
# Query data using Sql statements
query="""SELECT salary, name
FROM nba_roster
WHERE salary != '--'
ORDER BY CAST(REPLACE(REPLACE(salary, '$', ''), ',','') AS INTEGER) DESC
LIMIT 1;"""
df = pd.read_sql(query, con=engine)
df

,salary,name
0,"$49,649,218",Player 50


Create an Evaluation

In [27]:
# Get data to use as seed
with open('data/gold-test-set.jsonl', 'r') as file:
    data = [json.loads(line) for line in file]

print(data)

[{'question': 'What is the 99th percentile salary in the NBA?', 'answer': '46741590', 'sql': "SELECT (CAST(REPLACE(REPLACE(SALARY, '$', ''), ',','') AS INTEGER)) as percentile FROM nba_roster WHERE SALARY!= '--' order by percentile limit 1 offset (select count(*) from nba_roster where SALARY != '--')*99/100-1;"}, {'question': 'What is the 75th percentile salary in the NBA?', 'answer': '13932008', 'sql': "SELECT (CAST(REPLACE(REPLACE(SALARY, '$', ''), ',','') AS INTEGER)) as percentile FROM nba_roster WHERE SALARY!= '--' order by percentile limit 1 offset (select count(*) from nba_roster where SALARY != '--')*75/100-1;"}, {'question': 'What is the 25th percentile salary in the NBA?', 'answer': '2413304', 'sql': "SELECT (CAST(REPLACE(REPLACE(SALARY, '$', ''), ',','') AS INTEGER)) as percentile FROM nba_roster WHERE SALARY!= '--' order by percentile limit 1 offset (select count(*) from nba_roster where SALARY != '--')*25/100-1;"}, {'question': 'What is the median weight in the NBA?', 'ans

In [52]:
def get_schema_ex():
    return """\
0|tean|TEXT
1|name|TEXT
2|jersey|TEXT
3|pos|TEXT
4|age|INT
5|ht|TEXT
6|wt|TEXT
7|college|TEXT
8|sagary|TEXT eg.
"""

In [54]:
question = "What is the median weight in the NBA?"

system = f"""You are an NBA analyst with 15 years of experience writing complex SQL queries. Consider the nba_roster table with the following schema:
{get_schema_ex()}
Write a sqlite query to answer the following question. Follow instructions exactly"""

prompt = make_llama_3_prompt(question, system)
print(prompt)

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

You are an NBA analyst with 15 years of experience writing complex SQL queries. Consider the nba_roster table with the following schema:
0|tean|TEXT
1|name|TEXT
2|jersey|TEXT
3|pos|TEXT
4|age|INT
5|ht|TEXT
6|wt|TEXT
7|college|TEXT
8|sagary|TEXT eg.

Write a sqlite query to answer the following question. Follow instructions exactly<|eot_id|><|start_header_id|>user<|end_header_id|>

What is the median weight in the NBA?<|eot_id|><|start_header_id|>assistant<|end_header_id|>




In [56]:
generated_query = llm.generate(prompt, output_type={"sqlite_query": "str"}, max_new_tokens=200)
print(generated_query)

df = pd.read_sql(generated_query["sqlite_query"], con=engine)
df

{'sqlite_query': 'SELECT AVG(wt) FROM nba_roster'}


,AVG(wt)
0,225.6


In [58]:
# Try Agent Reflection to see if that can improve the query
reflection = f"Question: {question}. Query: {generated_query['sqlite_query']}. This query is invalid (gets the error Execution failed on sql 'SELECT AVG(CAST(SUBSTR(WT, INSTR(WT,'') + 1) AS INTEGER) FROM nba_roster WHERE WT IS NOT NULL': near \"FROM\": syntax error), so it cannot answer the question. Write a corrected sqlite query."
reflection_prompt = make_llama_3_prompt(reflection, system)
print(reflection_prompt)

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

You are an NBA analyst with 15 years of experience writing complex SQL queries. Consider the nba_roster table with the following schema:
0|tean|TEXT
1|name|TEXT
2|jersey|TEXT
3|pos|TEXT
4|age|INT
5|ht|TEXT
6|wt|TEXT
7|college|TEXT
8|sagary|TEXT eg.

Write a sqlite query to answer the following question. Follow instructions exactly<|eot_id|><|start_header_id|>user<|end_header_id|>

Question: What is the median weight in the NBA?. Query: SELECT AVG(wt) FROM nba_roster. This query is invalid (gets the error Execution failed on sql 'SELECT AVG(CAST(SUBSTR(WT, INSTR(WT,'') + 1) AS INTEGER) FROM nba_roster WHERE WT IS NOT NULL': near "FROM": syntax error), so it cannot answer the question. Write a corrected sqlite query.<|eot_id|><|start_header_id|>assistant<|end_header_id|>




In [59]:
reflection_query = llm.generate(reflection_prompt, output_type={"sqlite_query": "str"}, max_new_tokens=200)
print(reflection_query)

try:
  df = pd.read_sql(reflection_query["sqlite_query"], con=engine)
  df
except Exception as e:
  print(e)

{'sqlite_query': "SELECT AVG(CAST(SUBSTR(WT, INSTR(WT,'') + 1) AS INTEGER) FROM nba_roster WHERE WT IS NOT NULL"}
Execution failed on sql 'SELECT AVG(CAST(SUBSTR(WT, INSTR(WT,'') + 1) AS INTEGER) FROM nba_roster WHERE WT IS NOT NULL': near "FROM": syntax error


In [60]:
# Look at the right answer
correct_sql = "select CAST(SUBSTR(WT, 1, INSTR(WT,' ')) as INTEGER) as percentile from nba_roster order by percentile limit 1 offset (select count(*) from nba_roster)/2;"
df_corrected = pd.read_sql(correct_sql, con=engine)
df_corrected

,percentile
0,0


In [62]:
# Evaluate over a larger dataset
path = "data/gold-test-set.jsonl"

with jsonlines.open(path) as reader:
  data = [obj for obj in reader]

In [63]:
datapoint = data[4]
datapoint

{'question': 'What is the average weight in the NBA?',
 'answer': '214.98',
 'sql': "SELECT AVG(CAST(SUBSTR(WT, 1, INSTR(WT,' ')) as INTEGER)) FROM nba_roster;"}

In [64]:
datapoint = data[7]
datapoint

{'question': 'Can you tell me how many players are in the NBA?',
 'answer': '600',
 'sql': 'select count(*) from nba_roster;'}

In [73]:
system = "You are an NBA analyst with 15 years of experience writing complex SQL queries.\n"
system += "Consider the nba_roster table with the following schema:\n"
system += get_schema_ex() + "\n"
system += (
    "Write a sqlite SQL query that would help you answer the following question:\n"
)

user = datapoint["question"]
prompt = make_llama_3_prompt(user, system)
generated_sql = llm.generate(prompt, output_type={"sqlite_query": "str"}, max_new_tokens=200)
print(generated_sql)
df = pd.read_sql(generated_sql['sqlite_query'], con=engine)
df

{'sqlite_query': 'SELECT COUNT(*) FROM nba_roster'}


,COUNT(*)
0,100


In [74]:
ref_sql = datapoint["sql"]
ref_df = pd.read_sql(ref_sql, con=engine)
ref_df

,count(*)
0,100


In [75]:
# Compare strings
str(df).lower() == str(ref_df).lower()

True

In [79]:
# Compare by using an LLM model
system = "Compare the following two dataframes. They are similar if they are almost identical, or if they convey the same information about the nba_roster dataset"
system += "Respond with valid JSON {'explanation' : str, 'similar' : bool}"

user = (
    f"========== Dataframe 1 =========\n{str(df).lower()}\n\n"
)
user += (
    f"========== Dataframe 2 =========\n{str(ref_df).lower()}\n\n"
)
user += f"Can you tell me if these dataframes are similar?"

llm_similarity_prompt = make_llama_3_prompt(user, system)
print(llm_similarity_prompt)
llm_similarity = llm.generate(llm_similarity_prompt, output_type={"explanation": "str", "similar": "bool"}, max_new_tokens=200)
llm_similarity

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Compare the following two dataframes. They are similar if they are almost identical, or if they convey the same information about the nba_roster datasetRespond with valid JSON {'explanation' : str, 'similar' : bool}<|eot_id|><|start_header_id|>user<|end_header_id|>

========== Dataframe 1 =========
   count(*)
0       100

========== Dataframe 2 =========
   count(*)
0       100

Can you tell me if these dataframes are similar?<|eot_id|><|start_header_id|>assistant<|end_header_id|>




{'explanation': 'Both dataframes have the same count of 100, indicating they have the same number of rows. This suggests that they are similar',
 'similar': True}

In [107]:
class Args:
  def __init__(
    self,
    max_examples=100,
    sql_model_name="meta-llama/Meta-Llama-3-8B-Instruct",
    gold_file_name="gold-test-set.jsonl",
    training_file_name="archive/generated_queries.jsonl",
    num_to_generate=10
  ):
    self.sql_model_name = sql_model_name
    self.max_examples = max_examples
    self.gold_file_name = gold_file_name
    self.training_file_name = training_file_name
    self.num_to_generate = num_to_generate

In [108]:
def load_gold_dataset(args):
  path = f"data/{args.gold_file_name}"
  
  with jsonlines.open(path) as reader:
    for i, obj in enumerate(reversed(list(reader))):
      if i >= args.max_examples:
        break
      yield PromptObject(prompt="", data=obj)

In [109]:
# Here's how to wrap that all up in a runnable class
class QueryStage(GenerationNode):
  def __init__(self, model_name):
    super().__init__(
      model_name=model_name,
      max_new_tokens=200,
    )

  def generate(
    self,
    prompt: Union[Iterator[PromptObject], AsyncIterator[PromptObject]],
    *args,
    **kwargs,
  ):
    results = super().generate(
      prompt,
      output_type={"sqlite_query": "str"},
      *args,
      **kwargs,
    )
    return results

  def postprocess(self, obj: PromptObject):
    # Run both the generated and reference (Gold Dataset) SQL queries
    # Assessing whether the SQL queries succeeded in hitting the database (not correctness yet!)
    query_succeeded = False

    try:
      logger.error(f"Running SQL query '{obj.response['sqlite_query']}'")
      obj.data["generated_query"] = obj.response["sqlite_query"]
      df = pd.read_sql(obj.response["sqlite_query"], con=engine)
      obj.data['df'] = df
      logger.error(f"Got data: {df}")
      query_succeeded = True

    except Exception as e:
      logger.error(f"Failed to run SQL query: {obj.response['sqlite_query']}")

    logger.info(f"Running reference SQL query '{obj.data['sql']}'")
    df = pd.read_sql(obj.data["sql"], con=engine)
    logger.info(f"Got data: {df}")
    obj.data['reference_df'] = df

    logger.info(f"For question: {obj.data['question']}")
    logger.info(f"For query: {obj.response['sqlite_query']}")

    obj.data["query_succeeded"] = query_succeeded

  def preprocess(self, obj: PromptObject):
    new_prompt = make_llama_3_prompt(**self.make_prompt(obj.data))
    obj.prompt = new_prompt

  def make_prompt(self, data: dict):
    system = "You are an NBA analyst with 15 years of experience writing complex SQL queries.\n"
    system += "Consider the nba_roster table with the following schema:\n"
    system += get_schema() + "\n"
    system += ("Write a sqlite SQL query that would help you answer the following question:\n")
    user = data["question"]
    return {
      "user": user,
      "system": system,
    }

In [80]:
# How to wrap compare in a class
class ScoreStage(GenerationNode):
  def __init__(self):
    super().__init__(
        model_name="meta-llama/Meta-Llama-3-8B-Instruct",
        max_new_tokens=150,
    )

  def generate(
    self,
    prompt: Union[Iterator[PromptObject], AsyncIterator[PromptObject]],
    *args,
    **kwargs,
  ):
    logger.debug("ScoreStage Generate")
    results = super().generate(
      prompt,
      output_type={"explanation": "str", "similar": ["true", "false"]},
      *args,
      **kwargs,
    )        
    logger.debug(f"ScoreStage Results {results}")

    return results

  def preprocess(self, obj: PromptObject):
    obj.prompt = make_llama_3_prompt(**self.make_prompt(obj))
    logger.info(f"Scoring Stage Prompt:\n{obj.prompt}")

  def postprocess(self, obj: PromptObject):
    logger.info(f"Postprocess")
    obj.data['is_matching'] = self.is_matching(obj.data, obj.response)
    obj.data['explanation'] = obj.response["explanation"]
    obj.data['similar'] = obj.response["similar"] == "true"


  def is_matching(self, data, response):
    return (str(data.get('df',"None")).lower() == str(data['reference_df']).lower() or response['similar'] == "true")

  def make_prompt(self, obj: PromptObject):
    # Your evaluation model compares SQL output from the generated and reference SQL queries, using another LLM in the pipeline
    system_prompt = "Compare the following two dataframes. They are similar if they are almost identical, or if they convey the same information about the nba_roster dataset"
    system_prompt += "Respond with valid JSON {'explanation' : str, 'similar' : bool}"
    user_prompt = (
      f"========== Dataframe 1 =========\n{str(obj.data.get('df','None')).lower()}\n\n"
    )
    user_prompt += (
      f"========== Dataframe 2 =========\n{str(obj.data['reference_df']).lower()}\n\n"
    )
    user_prompt += f"Can you tell me if these dataframes are similar?"
    return {
      "system": system_prompt,
      "user": user_prompt
    }

In [83]:
class EvaluationPipeline(GenerationPipeline):
  def __init__(self, args):
    super().__init__()
    self.query_stage = QueryStage(args.sql_model_name)
    self.score_stage = ScoreStage()

  def forward(self, x):
    x = self.query_stage(x)
    x = self.score_stage(x)
    return x

In [93]:
async def run_eval(dataset, args):
  results = await run_evaluation_pipeline(dataset, args)
  print("Total results:", len(results))
  return results

async def run_evaluation_pipeline(dataset, args):
  results = EvaluationPipeline(args).call(dataset)
  result_list = []

  pbar = tqdm(desc="Saving results", unit=" results")
  async for result in results:
    result_list.append(result)
    pbar.update()
  return result_list

In [100]:
def save_eval_results(results, args):
  base_path = "./data/results"
  now = datetime.now().strftime("%Y_%m_%d_%H_%M_%S")
  experiment_name = f"nba_sql_pipeline_{now}"
  experiment_dir = os.path.join(base_path, experiment_name)
  os.makedirs(os.path.join(base_path, experiment_name))

  # Write args to file
  args_file_name = f"{experiment_dir}/args.txt"
  with open(args_file_name, "w") as writer:
    pprint(args.__dict__, writer)

  def is_correct(r):
    if ((r.data["query_succeeded"] and r.data['is_matching']) or r.data["generated_query"] == r.data['sql']):
      return True
    return False

  # Write sql results and errors to file
  results_file_name = f"{experiment_dir}/sql_results.jsonl"
  with jsonlines.open(results_file_name, "w") as writer:
    for result in results:
      if not is_correct(result):
        continue
      writer.write({
        "question": result.data['question'],
        "query": result.data["generated_query"],
        "query_succeeded": result.data["query_succeeded"],
        "reference_sql": result.data['sql'],
        "df": str(result.data.get('df', 'None')),
        "reference_df": str(result.data['reference_df']),
        "is_matching": result.data['is_matching'],
        "similar": result.data['similar'],
      })

  results_file_name = f"{experiment_dir}/sql_errors.jsonl"
  with jsonlines.open(results_file_name, "w") as writer:
    for result in results:
      if is_correct(result):
        continue
      writer.write({
        "question": result.data['question'],
        "query": result.data["generated_query"],
        "query_succeeded": result.data["query_succeeded"],
        "df": str(result.data.get('df', 'None')),
        "reference_df": str(result.data['reference_df']),
        "is_matching": result.data['is_matching'],
        "similar": result.data['similar'],
      })

  # Write statistics to file
  average_sql_succeeded = sum([result.data["query_succeeded"] for result in results]) / len(results)
  average_correct = sum([result.data["query_succeeded"] and result.data['is_matching'] for result in results]) / len(results)

  file_name = f"{experiment_dir}/summary.txt"
  with open(file_name, "w") as writer:
    print(f"Total size of eval dataset: {len(results)}", file=writer)
    print(f"Total size of eval dataset: {len(results)}")
    print(f"Percent Valid SQL Syntax: {average_sql_succeeded*100}", file=writer)
    print(f"Percent Valid SQL Syntax: {average_sql_succeeded*100}")
    print(f"Percent Correct SQL Query: {average_correct*100}", file=writer)
    print(f"Percent Correct SQL Query: {average_correct*100}")

In [110]:
args = Args()
dataset = load_gold_dataset(args)
results = await run_eval(dataset, args)
save_eval_results(results, args)



Running SQL query 'SELECT AVG(CAST(SUBSTR(SALARY, 2) AS INTEGER) FROM nba_roster WHERE POS = 'PF' AND SALARY!= '--';'
Failed to run SQL query: SELECT AVG(CAST(SUBSTR(SALARY, 2) AS INTEGER) FROM nba_roster WHERE POS = 'PF' AND SALARY!= '--';
Running SQL query 'SELECT Team, AVG(AGE) AS Average_Age FROM nba_roster GROUP BY Team ORDER BY Average_Age DESC LIMIT 5'
Got data:                     team  Average_Age
0  Golden State Warriors    31.937500
1        Milwaukee Bucks    30.833333
2     Los Angeles Lakers    30.181818
3       Dallas Mavericks    29.750000
4          Brooklyn Nets    29.750000
Running SQL query 'SELECT AVG(AGE) FROM nba_roster WHERE Team = 'Miami Heat';'
Got data:   AVG(AGE)
0     None
Running SQL query 'SELECT AVG(AGE) FROM nba_roster'
Got data:    AVG(AGE)
0     30.39
Running SQL query 'SELECT AVG(AGE) FROM nba_roster WHERE Team = 'Portland Trail Blazers';'
Got data:   AVG(AGE)
0     None
Running SQL query 'SELECT SALARY FROM nba_roster WHERE NAME = 'Marcus Smart';'

Total results: 20
Total size of eval dataset: 20
Percent Valid SQL Syntax: 55.00000000000001
Percent Correct SQL Query: 35.0
